# Process Data Collection and Signal State Ouput --- Merge the 2
Created by: Apoorb  
Date: Dec 3, 2019

## Load Libraries

In [70]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
os.chdir(r'C:\Users\abibeka\OneDrive - Kittelson & Associates, Inc\Documents\HCM-CAV-Pooled-Fund\VissimBaseModel')
os.getcwd()

'C:\\Users\\abibeka\\OneDrive - Kittelson & Associates, Inc\\Documents\\HCM-CAV-Pooled-Fund\\VissimBaseModel'

## Process Signal Data

Read the Signal State Data

In [71]:
SigDat = pd.read_csv('VissimBaseModel_010.lsa', delimiter =';',header=None,skiprows= 19)
SigDat.drop(columns=[1,2,6,7,8],inplace=True)
SigDat.columns = ['SimSec','PhaseNum','NewState','DurOldState']
SigDat.head()

,SimSec,PhaseNum,NewState,DurOldState
0,0.2,1,green,0.2
1,0.2,5,green,0.2
2,15.2,1,amber,15.0
3,15.2,5,amber,15.0
4,19.2,1,red,4.0


Keep only Phase 2 and 5.   
Keep Only New state of 'green' and 'amber' : green start and green end

In [72]:
SigDat_Nw = SigDat.copy()
def RenamePhase(x):
    y= "Null"
    if(x=="green"):
        y="Red"
    elif (x=="amber"):
        y="green"
    elif (x=="red"):
        y = 'amber'
    else : ""
    return(y)
SigDat_Nw.loc[:,'OldState'] = SigDat_Nw.NewState.apply(lambda x: RenamePhase(x.strip()))
SigDat_Nw.loc[:,'NewState'] = SigDat_Nw.NewState.str.strip()
SigDat_Nw = SigDat_Nw[SigDat_Nw.PhaseNum.isin([2,5])]
SigDat_Nw = SigDat_Nw[SigDat_Nw.NewState.isin(['green','amber'])]
SigDat_Nw.head()

,SimSec,PhaseNum,NewState,DurOldState,OldState
1,0.2,5,green,0.2,Red
3,15.2,5,amber,15.0,green
6,20.2,2,green,20.2,Red
8,57.2,2,amber,37.0,green
25,100.2,5,green,81.0,Red


Add new labels of green start and green end

In [73]:
SigDat_Nw  = SigDat_Nw[['SimSec','PhaseNum','NewState']]
SigDat_Nw.loc[:,'GrStat'] = SigDat_Nw.NewState.apply(lambda x: 'G_st' if x=="green" else "G_end")
SigDat_Nw.sort_values('SimSec',inplace=True)
SigDat_Nw.reset_index(drop=True,inplace=True)
SigDat_Nw.head()
#SigDat_Nw.tail()

,SimSec,PhaseNum,NewState,GrStat
0,0.2,5,green,G_st
1,15.2,5,amber,G_end
2,20.2,2,green,G_st
3,57.2,2,amber,G_end
4,100.2,5,green,G_st


In [74]:
np.ceil((SigDat_Nw.index.values+1) /4) #Cycle numbers if 1st row is kept

array([ 1.,  1.,  1.,  1.,  2.,  2.,  2.,  2.,  3.,  3.,  3.,  3.,  4.,
        4.,  4.,  4.,  5.,  5.,  5.,  5.,  6.,  6.,  6.,  6.,  7.,  7.,
        7.,  7.,  8.,  8.,  8.,  8.,  9.,  9.,  9.,  9., 10., 10., 10.,
       10., 11., 11., 11., 11., 12., 12., 12., 12., 13., 13., 13., 13.,
       14., 14., 14., 14., 15., 15., 15., 15., 16., 16., 16., 16., 17.,
       17., 17., 17., 18., 18., 18., 18., 19., 19., 19., 19., 20., 20.,
       20., 20., 21., 21., 21., 21.])

In [75]:
np.ceil((SigDat_Nw.index.values) /4) #Cycle numbers if 1st row is deleted

array([ 0.,  1.,  1.,  1.,  1.,  2.,  2.,  2.,  2.,  3.,  3.,  3.,  3.,
        4.,  4.,  4.,  4.,  5.,  5.,  5.,  5.,  6.,  6.,  6.,  6.,  7.,
        7.,  7.,  7.,  8.,  8.,  8.,  8.,  9.,  9.,  9.,  9., 10., 10.,
       10., 10., 11., 11., 11., 11., 12., 12., 12., 12., 13., 13., 13.,
       13., 14., 14., 14., 14., 15., 15., 15., 15., 16., 16., 16., 16.,
       17., 17., 17., 17., 18., 18., 18., 18., 19., 19., 19., 19., 20.,
       20., 20., 20., 21., 21., 21.])

Create Green Start, End Pairs : Call them CycNum

In [76]:
if(SigDat_Nw.GrStat.values[0] == 'G_st'):
    SigDat_Nw.loc[:,'CycNum'] = np.ceil((SigDat_Nw.index.values+1) /4)
else:
    SigDat_Nw.drop(index = 0,inplace =True)
    SigDat_Nw.loc[:,'CycNum'] = np.ceil((SigDat_Nw.index.values) /4)

In [77]:
SigDat_Nw.sort_values('SimSec',inplace=True)

Get start and end of phase 1 and 5 within each cycle

In [80]:
SigDat_Nw_clean = pd.pivot_table(SigDat_Nw,values = 'SimSec',index=['CycNum','PhaseNum'],columns='GrStat').reset_index()
SigDat_Nw_clean.sort_values(['G_st'],inplace=True)
SigDat_Nw_clean.head()
SigDat_Nw_clean.tail()

GrStat,CycNum,PhaseNum,G_end,G_st
36,19.0,2,1857.2,1820.2
39,20.0,5,1915.2,1900.2
38,20.0,2,1957.2,1920.2
41,21.0,5,2015.2,2000.2
40,21.0,2,2057.2,2020.2


## Process Data Collection Point Output

In [81]:
DataColDat = pd.read_csv('VissimBaseModel_010.mer', delimiter =';',skiprows= 11)
DataColDat = DataColDat.iloc[:,0:-1]
DataColDat.columns = ['Lane','t_Entry','t_Exit','VehNo','VehType','Del','V_mph','Acc_ftsec2','Del2','Del3','tQueue','VehLen_ft']
DataColDat = DataColDat[['Lane','t_Entry','t_Exit','VehNo','VehType','V_mph','Acc_ftsec2','tQueue','VehLen_ft']]
DataColDat_Sub = DataColDat[DataColDat.t_Entry!=-1]
DataColDat_Sub = DataColDat_Sub[['Lane','t_Entry','tQueue','VehLen_ft','V_mph','Acc_ftsec2']]

In [82]:
DataColDat_Sub.shape[0]
DataColDat_Sub.head()
DataColDat_Sub.sort_values('t_Entry',inplace=True)

Use pandas.merge_asof for merging t_Entry in data collection results to the nearest phase start time.   
Similar to infimum ~ Greatest lower bound  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge_asof.html  
A “backward” search selects the last row in the right DataFrame whose ‘on’ key is less than or equal to the left’s key.  

In [83]:
CombData_Ln_Dict = {}
for Ln in [1,2]:
    CombData_Ln_Dict[Ln] = pd.merge_asof(DataColDat_Sub[DataColDat_Sub.Lane==Ln],SigDat_Nw_clean, left_on='t_Entry', right_on= 'G_st',direction = 'backward')
    # Check the vehicles that eneter the intersection during Amber Indication 
    CombData_Ln_Dict[Ln].loc[:,'VehinAmber'] = CombData_Ln_Dict[Ln].t_Entry > CombData_Ln_Dict[Ln].G_end
CombData = pd.concat([CombData_Ln_Dict[1],CombData_Ln_Dict[2]])
#CombData_Ln_Dict[Ln][CombData_Ln_Dict[Ln].VehinAmber]

In [84]:
Ln=1
CombData_Ln_Dict[Ln][CombData_Ln_Dict[Ln].VehinAmber]

print(CombData_Ln_Dict[Ln].PhaseNum.value_counts())
Ln=2
print(CombData_Ln_Dict[Ln].PhaseNum.value_counts())

2    289
Name: PhaseNum, dtype: int64
2    351
Name: PhaseNum, dtype: int64


In [85]:
CombData.sort_values(['CycNum','Lane'],inplace=True)
CombData.loc[:,'VehNum'] = np.hstack(CombData.groupby(['CycNum','Lane'])['t_Entry'].apply(lambda x: np.arange(1,len(x)+1)).values)

In [86]:
CombData.loc[:,'dummy'] = 1
CombData.loc[:,'VehNum2'] =  CombData.groupby(['CycNum','Lane']).dummy.cumsum().values
CombData.loc[:,'Error'] = CombData.loc[:,'VehNum2'] - CombData.loc[:,'VehNum']
CombData.loc[:,'Error'].value_counts()
CombData.reset_index(drop = True, inplace=True)
CombData_Backup = CombData.copy()


In [87]:
CombData = CombData[['CycNum','Lane','tQueue','t_Entry','G_st','G_end','VehNum','PhaseNum']]
# Get the timestamp for the 4th veh and the last vehicle in the queue
mask = (CombData.VehNum >4) & (CombData.tQueue> 0)
print(mask.sum()) # of values kept
print(mask.shape[0]) #orignal size
CombData = CombData[mask]

475
640


In [88]:
CombData = CombData.groupby(['CycNum','Lane']).agg({'t_Entry':['min','max'],'VehNum':['min','max']})

In [89]:
CombData.columns = ['_'.join(col).strip() for col in CombData.columns.values]

In [90]:
CombData.loc[:,'AvgHeadway'] = (CombData.t_Entry_max - CombData.t_Entry_min)/(CombData.VehNum_max - CombData.VehNum_min)

In [91]:
CombData.AvgHeadway.describe()
idx = pd.IndexSlice

#Average headway lane 1
print(3600/CombData.loc[idx[:,1],'AvgHeadway'].mean())
#Average headway lane 2
print(3600/CombData.loc[idx[:,2],'AvgHeadway'].mean())

1485.5749328414863
1859.0198215100847


In [92]:
CombData.to_csv('CheckDat.csv')

In [93]:
VisualData = CombData_Backup.copy()
VisualData.loc[:,'Headway'] = VisualData.groupby(['CycNum','Lane'])['t_Entry'].diff()
VisualData.dropna(inplace=True)
VisualData = VisualData[['CycNum','Lane','t_Entry','G_st','G_end','VehNum2','Headway','tQueue']]

In [100]:
# Delay should in general keep on decreasing after the start of green
print( VisualData.groupby(['CycNum','Lane'])['tQueue'].diff(-1).describe())
VisualData.groupby(['CycNum','Lane'])['tQueue'].diff(-1).values

count    568.000000
mean       3.352641
std        5.488233
min      -25.400000
25%        0.100000
50%        2.300000
75%        5.900000
max       30.900000
Name: tQueue, dtype: float64


array([  6. ,   4.1,   6.2,   1. ,  -0.5,  -0.5,  16.3,  20.5,   6. ,
         1.6,  -0.4,   6.6,   0.2,   1.4,   2.7,   nan,  12.2,   0.2,
         1.8,   1.3,   4. ,   0.1,  -0.5,  30.9,   8.4,   4.3,   0.3,
         1.1,  -0.2,  -0.5,   2. ,  -1.7,  -0.2,   3.4,   nan,   4.1,
         2.2,   7.2,   6.1,   0.3,   0.8,  17. ,  -0.2,  -1.9,   0.3,
        -0.7,  -0.3,   7.9,   nan,   0.8,  11.1,   3.3,  12. ,   1.1,
         2.4,   4.4,   4.3,   1.5,  -1.8,   1.2,  -1.3,   0.5,  -1.2,
         7.1,   0.3,   1.7,   9. ,   nan,   4.6,  10.5,   0. ,   2.7,
         5.7,   2.5,  -0.1,   0.9,  -0.6,   1.8,  -0.2,   3.1,   2.7,
         nan,   5.1,  10.8,   0.4,   3.9,   2.4,   1.3,   2.4,   0.8,
        -0.1,   0.7,  -0.4,   0.4,  -0.1,   0. ,  -5.9,   3.7,  -0.7,
         7.5,   nan,   3.3,   6.7,   2.4,   5.1,   0.8,   0.2,  -0.1,
         0.1,  -0.7,   5.9,  10.2,   4.2,  12.2,  -1.1,  -1. ,  -1.6,
         nan,   2.3,   2.8,   8.6,  10.8,   1. ,   4.7,  -0.4,   1. ,
        -0.1,   0.1,

In [94]:
Cycle_St_End = VisualData.groupby('CycNum')['G_st','G_end'].first()
list_shade = []
for index, row in Cycle_St_End.iterrows():
    list_shade.append(
    go.layout.Shape(
    type = 'rect',
    xref='x',
    yref ='paper',
    x0= row['G_st'],
    y0 = 0,
    x1 = row['G_end'],
    y1= 1,
    fillcolor = "LightSalmon",
    opacity = 0.5,
    layer = "below",
    line_width = 0)
    )

In [95]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=VisualData.loc[VisualData.Lane==1,'t_Entry'], y= VisualData.loc[VisualData.Lane==2,'tQueue'],
                    mode='markers', name='markers'))
fig.update_layout(plot_bgcolor = 'rgb(220,220,220)', shapes= list_shade)
fig.show()

In [68]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=VisualData.loc[VisualData.Lane==1,'t_Entry'], y= VisualData.loc[VisualData.Lane==2,'Headway'],
                    mode='markers', name='markers'))
fig.update_layout(plot_bgcolor = 'rgb(220,220,220)', shapes= list_shade)
fig.show()